In [1]:
import altair as alt
import os
import pandas as pd
import glob
import tensorflow as tf

2022-10-03 16:53:51.384917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 16:53:51.769271: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 16:53:53.749061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-03 16:53:53.749232: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [2]:
summary_iterator = tf.compat.v1.train.summary_iterator

def get_section_results(file, xtag='Train_EnvstepsSoFar', ytag='Eval_AverageReturn'):
    """
        requires tensorflow==1.12.0
    """
    X = []
    Y = []
    for e in summary_iterator(file):
        for v in e.summary.value:
            if v.tag == xtag:
                X.append(v.simple_value)
            elif v.tag == ytag:
                Y.append(v.simple_value)
    return X, Y
logdir = '../../data/q2_pg_q2*/events*'
glob.glob(logdir)


def make_dataframe(eventfiles, xtag='Train_EnvstepsSoFar', ytag='Eval_AverageReturn'):
    dfs = []
    for eventfile in eventfiles:
        dirname = os.path.basename(os.path.dirname(eventfile))
        X, Y = get_section_results(eventfile, xtag, ytag)
        n = len(X)
        assert len(X) == len(Y), (len(X), len(Y), eventfile, X, Y)
        log_df = pd.DataFrame.from_dict({xtag: X, ytag: Y, 'exp': [dirname] * n})
        dfs.append(log_df)
    
    return pd.concat(dfs)

# Q1

In [4]:
logdir = '../../data/q1_*/events*'
# eventfiles = glob.glob(logdir)
# xtag = 'Train_EnvstepsSoFar'
# ytag = 'Train_AverageReturn'
# df_q1  = make_dataframe(eventfiles, xtag=xtag, ytag=ytag)

# chart = alt.Chart(df_q1).mark_line().encode(
#     x=xtag,
#     y=ytag,
#     color='exp'
# )
# chart = chart.configure_legend(
#     labelLimit=1000
# )
# chart

In [5]:
%load_ext tensorboard
%tensorboard --logdir {logdir}